# Dimer choices

In this notebook we'll use some quick visualization tools to try and determine which dimers we will investigate in order to understand the influence of substituting certain elements on the dimerization.

In [4]:
# Notebook Header

%matplotlib inline
import imolecule
import os
import pybat.core as pb

from ipywidgets import interact_manual, fixed, IntSlider

cathode_file = "/Users/mbercx/Dropbox/Phd/projects/batteries/data/Li2MnO3_Codop/0.5/o1-aaa-1a-fm/222/" \
    + "O1-Li0.5MnO3_Codop-1a_222.json"

def visualize_dimer(cathode_file, dimer_index1, dimer_index2):
    """
    Visualizes the dimer using 
    """
    
    cat = pb.Cathode.from_file(cathode_file)
       
    cat.replace(dimer_index1, "F")
    cat.replace(dimer_index2, "F")
    cat.change_site_distance([dimer_index1, dimer_index2], 1.4)
    cat.to("cif", "temp.cif")
    imolecule.draw("temp.cif", element_properties={
        "Li": { "color": 0x66ff66, "radius": 0.85 },
        "O": { "color": 0xff0000, "radius": 0.7 },
        "F": { "color": 0xffcc00, "radius": 0.7 },
        "Mn": { "color": 0xf00fff, "radius": 1.35 },
        "Co": { "color": 0x0000ff, "radius": 1.3},
        "Mo": { "color": 0x0000ff, "radius": 1.3},
        "Sn": { "color": 0x0000ff, "radius": 1.3},
        "V": { "color": 0x0000ff, "radius": 1.3}})
    
def visualize_dimer_dir(dimer_directory):
    """
    Visualizes the dimer using 
    """
    
    if not os.path.isdir(dimer_directory):
        print("Input path is not a directory.")
        return
    
    cat = pb.Cathode.from_file(os.path.join(dimer_directory, "final", "final_cathode.json"))
    dimer_indices = [int(number) for number in os.path.basename(dimer_directory.strip("/")).split("_")[1:]]
    
    if dimer_indices == []:
        print("Could not extract dimer indices from directory.")
    else:         
        cat.replace(dimer_indices[0], "F")
        cat.replace(dimer_indices[1], "F")
        cat.to("cif", "temp.cif")
        imolecule.draw("temp.cif", element_properties={
            "Li": { "color": 0x66ff66, "radius": 0.85 },
            "O": { "color": 0xff0000, "radius": 0.7 },
            "F": { "color": 0xffcc00, "radius": 0.7 },
            "Mn": { "color": 0xf00fff, "radius": 1.35 },
            "Co": { "color": 0x0000ff, "radius": 1.3},
            "Mo": { "color": 0x0000ff, "radius": 1.3},
            "Sn": { "color": 0x0000ff, "radius": 1.3},
            "V": { "color": 0x0000ff, "radius": 1.3}})

In [2]:
interact_manual(visualize_dimer, cathode_file=fixed(cathode_file), 
         dimer_index1=IntSlider(value=48, min=48, max=95), 
         dimer_index2=IntSlider(value=49, min=48, max=95));

interactive(children=(IntSlider(value=48, description='dimer_index1', max=95, min=48), IntSlider(value=49, des…

The interactive tool above can be used to visualize various choices of dimers. Based on the dimers we found to be thermodynamically viable for Li2MnO3, we start by adding the following two dimers to the list of dimers that should be investigated:

* `49-93`: This dimer corresponds to dimer E in our MRS advances paper.
* `55-86`: This dimer corresponds to dimer A in our MRS advances paper.

Note that the dimers are defined by the internal indices of the `Cathode` object. Basically, we want to check if substituting various elements has an influence on the dimerization of oxygen pairs with a similar immediate environment as we had for the thermodynamically viable dimers in O1-Li2MnO3. Moreover, we also want to check the oxygen pairs of the octahedron of the substituted element:

In [3]:
cat = pb.LiRichCathode.from_file(cathode_file)
print(cat)

Full Formula (Li8 Mn15 Co1 O48)
Reduced Formula: Li8Mn15CoO48
abc   :   9.997196   9.997127   8.230322
angles:  89.999665  90.001539 120.001613
Sites (96)
  #  #VESTA    SP           a         b         c    magmom
---  --------  ----  --------  --------  --------  --------
  0  1         Li    0         0         0              0
  1  2         Li    1         0         0.5            0
  2  3         Li    0         0.5       0              0
  3  4         Li    0         0.5       0.5            0
  4  5         Li    0.5       1         1              0
  5  6         Li    0.5       0         0.5            0
  6  7         Li    0.5       0.5       0              0
  7  8         Li    0.5       0.5       0.5            0
  8  -         Vac   1         0         0.25           0
  9  -         Vac   0         0         0.75           0
 10  -         Vac   0         0.5       0.25           0
 11  -         Vac   0         0.5       0.75           0
 12  -         Vac   0.5     

In [8]:
cat.list_noneq_dimers(32) # Look for the nonequivalent dimers around the Co site.

[[(58, 64), (58, 48), (64, 48), (88, 80), (88, 74), (80, 74)],
 [(58, 88), (64, 80), (48, 74)],
 [(58, 74), (64, 88), (80, 48)]]

As we can see, there are three lists of equivalent dimers which are inequivalent versus elements of the other lists (if that makes sense). We'll have to choose one of each and add them to the list of dimers to be studied. We've picked `88-80`, `64-80` and `48-80`. Finally, we also want to investigate the dimer across the lithium layer, with one oxygen of the pair neighboring the substituted element, as this dimer was found to be the most optimal for Li2MnO3. We choose dimer `49-80`. Our final list of dimers is:

1. `49-93`
2. `55-86`
3. `88-80`
4. `64-80`
5. `48-80`
6. `49-80`

Feel free to check any of these dimer using the tool above.

## Visualizing the final dimers

In [5]:
data_dir = "/users/mbercx/Dropbox/Phd/projects/batteries/data/Li2MnO3_Codop/0.5/o1-aaa-1a-fm/222"
pbeu_dir = "pbeu_Mn3.9Co5.1_dimers"
scan_dir = "scan_dimers"

In [6]:
visualize_dimer_dir(os.path.join(data_dir, pbeu_dir, "dimer_80_48"))